In [2]:
import gspread
from google.oauth2.service_account import Credentials

import pandas as pd


use water_env_broad_scale_voting_sheet in 2024-12-12T1240EDT Google Drive folder,
https://docs.google.com/spreadsheets/d/1lMjVxyADmZM1rZI7Qo99RnN6V5K8B8GjAevVoKENViw/edit?gid=1038036384#gid=1038036384

make sure that's shared with env-context-voting-sheets@env-context-voting-sheets.iam.gserviceaccount.com

In [3]:
output_file = "post_google_sheets_water_env_medium.tsv"

Get voting sheet directly from Google Sheets. Requires a credentials file, which must be requested from MAM.

In [4]:
# 1. Replace with the path to your downloaded JSON key file
SERVICE_ACCOUNT_FILE = '../../env-context-voting-sheets-29d0d970ca1d.json'

In [5]:
# Consolidated_soil-env-local-scale-evidence-table
SPREADSHEET_ID = "1jMhQ6QK_50jK1MoJq8_raEfrW1FgYQJbB5r1D2SjFQ0"

In [6]:
sheet_name = 'water_env_medium_voting_sheet'

In [7]:
# Set up credentials
SCOPES = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

In [8]:
creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

In [9]:
gc = gspread.authorize(creds)

In [13]:
# Open the spreadsheet
spreadsheet = gc.open_by_key(SPREADSHEET_ID) 

In [14]:
sheet = spreadsheet.worksheet(sheet_name)

In [15]:
# Get data as a list of lists
data = sheet.get_all_values()

In [16]:
# Convert to DataFrame
df = pd.DataFrame(data[1:], columns=data[0]) 

In [17]:
# make a dataframe that has the columns curie and label. only include the rows where the include_in_enum column value is true
votes_df = df[df['include_in_enum'] == 'TRUE'][['curie', 'label']]

In [18]:
votes_df.columns = ['id', 'label']

In [19]:
votes_df

,id,label
2,ENVO:04000007,lake water
3,ENVO:03605001,epilithon
4,ENVO:00002042,surface water
6,ENVO:03605006,stream water
7,ENVO:03605004,epipsammon
...,...,...
2544,ENVO:03600081,cloud water
2627,ENVO:00003065,distilled water
2700,ENVO:03000005,erosionally enriched glacial ice
2751,ENVO:01001512,congelation sea ice


In [20]:
votes_df.to_csv(output_file, index=False, sep="\t")